In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
cfmat1 = np.load("Part 1 Output/single_frame_confusion_matrix.npy")
probability1 = np.load("Part 1 Output/single_frame_prob_distribution.npy")
prediction1 = pd.read_csv("Part 1 Output/SingleFrame_predictions.csv")
test1 = pd.read_csv("Part 1 Output/SingleFrame_test.csv")
checkpoint1 = torch.load("Part 1 Output/Singleframe_Video_AR.checkpoint.pth.tar")

In [3]:
cfmat2 = np.load("Part 2 Output/sequence_frames_Subseq_confusion_matrix.npy")
probability2 = np.load("Part 2 Output/sequence_frames_prob_distribution.npy")
prediction2 = pd.read_csv("Part 2 Output/SequenceFrames_Subseq_predictions.csv")
test2 = pd.read_csv("Part 2 Output/SequenceFrames_Subseq_test.csv")
checkpoint2 = torch.load("Part 2 Output/Sequenceframes_Video_AR.checkpoint.pth.tar")

In [13]:
prediction1.columns

Index(['Unnamed: 0', 'Labels', 'Predictions'], dtype='object')

In [30]:
weight = 0.1
probability3 = weight*probability1 + (1-weight)*probability2
prediction3 = prediction2
pred_3 = np.argsort(-probability3, axis=1)
acctop1_3 = 0
acctop5_3 = 0
acctop10_3 = 0
for j in range(probability3.shape[0]):
    label = prediction3.Labels[j]
    prediction3.Predictions[j] = pred_3[j, 0]
    if label == pred_3[j, 0]:
        acctop1_3 += 1
    if label in pred_3[j, :5]:
        acctop5_3 += 1
    if label in pred_3[j, :10]:
        acctop10_3 += 1
acctop1_3/probability3.shape[0], acctop5_3/probability3.shape[0], acctop10_3/probability3.shape[0]

(0.8220988633359767, 0.9603489294210944, 0.9785884218873909)

In [48]:
cfmat3 = np.zeros((101,101), dtype=np.float32)
prediction3_mat = np.array(prediction3)
for j in range(len(prediction3_mat)):
    cfmat3[int(prediction3_mat[j][1]), int(prediction3_mat[j][2])] += 1

In [53]:
for j in range(len(cfmat3)):
    cfmat3[j] = cfmat3[j] / sum(cfmat3[j])

In [55]:
acc_classes_3 = np.diag(cfmat3)
acc_classes_ind_3 = np.argsort(acc_classes_3)
#best_perf10_1, worst_perf10_1 = acc_classes_ind_1[-10:], acc_classes_ind_1[:10]
best_perf10_3, worst_perf10_3 = np.where(acc_classes_3==1)[0], acc_classes_ind_3[:10]

In [56]:
classInd = pd.read_csv("classInd.txt", sep=" ", header=None)
classInd.columns = ["Index","ClassNames"]
(best_perf10_classes_3, worst_perf10_classes_3) = (list(classInd.ClassNames[best_perf10_3]), list(classInd.ClassNames[worst_perf10_3]))
(best_perf10_classes_3, worst_perf10_classes_3)

(['BabyCrawling',
  'Billiards',
  'BoxingSpeedBag',
  'Fencing',
  'FloorGymnastics',
  'HorseRiding',
  'JumpingJack',
  'Knitting',
  'PlayingDhol',
  'PlayingGuitar',
  'PlayingPiano',
  'PlayingTabla',
  'PoleVault',
  'RockClimbingIndoor',
  'Skijet',
  'SoccerPenalty',
  'StillRings',
  'VolleyballSpiking'],
 ['Nunchucks',
  'YoYo',
  'Lunges',
  'JavelinThrow',
  'HandstandWalking',
  'Shotput',
  'MoppingFloor',
  'CricketBowling',
  'PizzaTossing',
  'SoccerJuggling'])

In [57]:
len(best_perf10_classes_3)

18

In [58]:
# Select the 10 pairs of most confused classes
allcfmat3 = cfmat3.reshape(-1)
diagcfmat3_ind = range(0, len(allcfmat3), 102)
allcfmat3_sortind = np.argsort(-allcfmat3)
offcfmat3_sortind = [x for x in allcfmat3_sortind if x not in diagcfmat3_ind]

cfmat3_confuse10 = np.array(offcfmat3_sortind[:10])
cfmat3_confuse10_label = cfmat3_confuse10 // 101
cfmat3_confuse10_pred = cfmat3_confuse10 % 101

cfmat3_confuse10_label, cfmat3_confuse10_pred
(cfmat3_confuse10_class_label, cfmat3_confuse10_class_pred) = (list(classInd.ClassNames[cfmat3_confuse10_label]), list(classInd.ClassNames[cfmat3_confuse10_pred]))
[[cfmat3_confuse10_class_label[i], cfmat3_confuse10_class_pred[i]] for i in range(len(cfmat3_confuse10_label))]

[['Haircut', 'BlowDryHair'],
 ['JavelinThrow', 'LongJump'],
 ['Rowing', 'Skijet'],
 ['YoYo', 'JugglingBalls'],
 ['FrontCrawl', 'BreastStroke'],
 ['MoppingFloor', 'HandstandWalking'],
 ['HighJump', 'LongJump'],
 ['CricketShot', 'CricketBowling'],
 ['BoxingPunchingBag', 'BoxingSpeedBag'],
 ['ApplyLipstick', 'BrushingTeeth']]

In [19]:
pred10_3[0,:]

array([ 91,  96,  48,  88,  97,  25,  41,   7,  93,  80,  83,  16,  10,
        82,  81,   9,  44,  55,  67,  38,  32,  40,   8,  87,  18,  72,
        27,  74,  76,  30,  75,  23,  42,  79,  70,  66,  31,  51,  52,
        22,   2,  28, 100,   5,  90,   6,  84,  43,  17,  39,  47,  29,
        35,  56,  92,  20,  21,  46,  54,  95,  45,   4,  78,  36,  50,
         1,  60,  73,  57,  69,  12,  89,  61,  68,   3,  58,  99,  85,
        98,  77,  26,  71,  62,  15,  86,  11,  14,  19,  13,  65,  59,
        63,  64,   0,  37,  49,  33,  53,  34,  24,  94], dtype=int64)

In [4]:
# Accuracies for two problems
acctop1_1, acctop5_1, acctop10_1 = test1.ACC_1[0], test1.ACC_5[0], test1.ACC_10[0]
acctop1_2, acctop5_2, acctop10_2 = test2.ACC_1[0], test2.ACC_5[0], test1.ACC_10[0]

(acctop1_1, acctop5_1, acctop10_1), (acctop1_2, acctop5_2, acctop10_2)

((0.7858842188739096, 0.936558287073751, 0.9669574411842452),
 (0.8300290774517579, 0.9598202484800424, 0.9669574411842452))

In [6]:
# Select 10 best and worst classes
acc_classes_1 = np.diag(cfmat1)
acc_classes_ind_1 = np.argsort(acc_classes_1)
#best_perf10_1, worst_perf10_1 = acc_classes_ind_1[-10:], acc_classes_ind_1[:10]
best_perf10_1, worst_perf10_1 = np.where(acc_classes_1==1)[0], acc_classes_ind_1[:10]
acc_classes_2 = np.diag(cfmat2)
acc_classes_ind_2 = np.argsort(acc_classes_2)
#best_perf10_2, worst_perf10_2 = acc_classes_ind_2[-10:], acc_classes_ind_2[:10]
best_perf10_2, worst_perf10_2 = np.where(acc_classes_2==1)[0], acc_classes_ind_2[:10]

(best_perf10_1, worst_perf10_1), (acc_classes_1[best_perf10_1], acc_classes_1[worst_perf10_1]), (best_perf10_2, worst_perf10_2), (acc_classes_2[best_perf10_2], acc_classes_2[worst_perf10_2])

((array([ 8, 11, 15, 18, 25, 29, 40, 41, 52, 59, 65, 67, 73, 75, 81, 82, 84,
         86, 87, 99], dtype=int64),
  array([ 47,  37, 100,  55,  14,  39,  31,  69,  98,  51], dtype=int64)),
 (array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1.], dtype=float32),
  array([0.        , 0.14705883, 0.19444445, 0.25714287, 0.26666668,
         0.2972973 , 0.35135135, 0.35714287, 0.4       , 0.4054054 ],
        dtype=float32)),
 (array([ 3,  4,  8, 11, 15, 17, 27, 29, 40, 41, 46, 49, 60, 62, 63, 65, 67,
         73, 81, 84, 85, 96, 98], dtype=int64),
  array([ 55,  51,  44, 100,  78,  37,  54,  83,  57,  39], dtype=int64)),
 (array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1.], dtype=float32),
  array([0.25714287, 0.27027026, 0.32258064, 0.33333334, 0.41304347,
         0.44117647, 0.44117647, 0.46153846, 0.4848485 , 0.4864865 ],
        dtype=float32)))

In [7]:
# Transform into classes format
classInd = pd.read_csv("classInd.txt", sep=" ", header=None)
classInd.columns = ["Index","ClassNames"]
(best_perf10_classes_1, worst_perf10_classes_1) = (list(classInd.ClassNames[best_perf10_1]), list(classInd.ClassNames[worst_perf10_1]))
(best_perf10_classes_1, worst_perf10_classes_1)

(['BasketballDunk',
  'Billiards',
  'Bowling',
  'BreastStroke',
  'Diving',
  'FloorGymnastics',
  'HorseRace',
  'HorseRiding',
  'MilitaryParade',
  'PlayingDaf',
  'PlayingTabla',
  'PoleVault',
  'RockClimbingIndoor',
  'Rowing',
  'Skijet',
  'SkyDiving',
  'SoccerPenalty',
  'SumoWrestling',
  'Surfing',
  'WritingOnBoard'],
 ['JumpRope',
  'HandstandWalking',
  'YoYo',
  'Nunchucks',
  'BodyWeightSquats',
  'HighJump',
  'FrontCrawl',
  'PullUps',
  'WallPushups',
  'Lunges'])

In [8]:
classInd = pd.read_csv("classInd.txt", sep=" ", header=None)
classInd.columns = ["Index","ClassNames"]
(best_perf10_classes_2, worst_perf10_classes_2) = (list(classInd.ClassNames[best_perf10_2]), list(classInd.ClassNames[worst_perf10_2]))
(best_perf10_classes_2, worst_perf10_classes_2)

(['BabyCrawling',
  'BalanceBeam',
  'BasketballDunk',
  'Billiards',
  'Bowling',
  'BoxingSpeedBag',
  'Fencing',
  'FloorGymnastics',
  'HorseRace',
  'HorseRiding',
  'JumpingJack',
  'Knitting',
  'PlayingDhol',
  'PlayingGuitar',
  'PlayingPiano',
  'PlayingTabla',
  'PoleVault',
  'RockClimbingIndoor',
  'Skijet',
  'SoccerPenalty',
  'StillRings',
  'VolleyballSpiking',
  'WallPushups'],
 ['Nunchucks',
  'Lunges',
  'JavelinThrow',
  'YoYo',
  'Shotput',
  'HandstandWalking',
  'MoppingFloor',
  'SoccerJuggling',
  'PizzaTossing',
  'HighJump'])

In [10]:
len(best_perf10_classes_1), len(best_perf10_classes_2)

(20, 23)

In [12]:
set(best_perf10_classes_2)-(set(best_perf10_classes_1).intersection(set(best_perf10_classes_2)))

{'BabyCrawling',
 'BalanceBeam',
 'BoxingSpeedBag',
 'Fencing',
 'JumpingJack',
 'Knitting',
 'PlayingDhol',
 'PlayingGuitar',
 'PlayingPiano',
 'StillRings',
 'VolleyballSpiking',
 'WallPushups'}

In [23]:
# Select the 10 pairs of most confused classes
allcfmat1 = cfmat1.reshape(-1)
diagcfmat1_ind = range(0, len(allcfmat1), 102)
allcfmat1_sortind = np.argsort(-allcfmat1)
offcfmat1_sortind = [x for x in allcfmat1_sortind if x not in diagcfmat1_ind]

cfmat1_confuse10 = np.array(offcfmat1_sortind[:10])
cfmat1_confuse10_label = cfmat1_confuse10 // 101
cfmat1_confuse10_pred = cfmat1_confuse10 % 101

cfmat1_confuse10_label, cfmat1_confuse10_pred

(array([31, 33, 68, 39, 19, 23, 37, 47, 35, 14], dtype=int64),
 array([18, 12, 56, 44, 77, 22, 71, 42, 92, 98], dtype=int64))

In [24]:
(cfmat1_confuse10_class_label, cfmat1_confuse10_class_pred) = (list(classInd.ClassNames[cfmat1_confuse10_label]), list(classInd.ClassNames[cfmat1_confuse10_pred]))
[[cfmat1_confuse10_class_label[i], cfmat1_confuse10_class_pred[i]] for i in range(len(cfmat1_confuse10_label))]

[['FrontCrawl', 'BreastStroke'],
 ['Haircut', 'BlowDryHair'],
 ['PommelHorse', 'ParallelBars'],
 ['HighJump', 'JavelinThrow'],
 ['BrushingTeeth', 'ShavingBeard'],
 ['CricketShot', 'CricketBowling'],
 ['HandstandWalking', 'PushUps'],
 ['JumpRope', 'HulaHoop'],
 ['HammerThrow', 'ThrowDiscus'],
 ['BodyWeightSquats', 'WallPushups']]

In [25]:
allcfmat2 = cfmat2.reshape(-1)
diagcfmat2_ind = range(0, len(allcfmat2), 102)
allcfmat2_sortind = np.argsort(-allcfmat2) # Sort the maximum
offcfmat2_sortind = [x for x in allcfmat2_sortind if x not in diagcfmat1_ind]

cfmat2_confuse10 = np.array(offcfmat2_sortind[:10])
cfmat2_confuse10_label = cfmat2_confuse10 // 101
cfmat2_confuse10_pred = cfmat2_confuse10 % 101

cfmat2_confuse10_label, cfmat2_confuse10_pred

(array([ 33,  44,  75, 100,  39,  31,  54,  16,  34,  47], dtype=int64),
 array([12, 50, 81, 45, 50, 18, 37, 17, 38, 46], dtype=int64))

In [26]:
(cfmat2_confuse10_class_label, cfmat2_confuse10_class_pred) = (list(classInd.ClassNames[cfmat2_confuse10_label]), list(classInd.ClassNames[cfmat2_confuse10_pred]))
[[cfmat2_confuse10_class_label[i], cfmat2_confuse10_class_pred[i]] for i in range(len(cfmat2_confuse10_label))]

[['Haircut', 'BlowDryHair'],
 ['JavelinThrow', 'LongJump'],
 ['Rowing', 'Skijet'],
 ['YoYo', 'JugglingBalls'],
 ['HighJump', 'LongJump'],
 ['FrontCrawl', 'BreastStroke'],
 ['MoppingFloor', 'HandstandWalking'],
 ['BoxingPunchingBag', 'BoxingSpeedBag'],
 ['Hammering', 'HeadMassage'],
 ['JumpRope', 'JumpingJack']]

In [13]:
prediction1.columns

Index(['Unnamed: 0', 'Labels', 'Predictions'], dtype='object')

In [24]:
a = prediction1.Labels==33
b = prediction1.Predictions==12
cc = prediction1[a & b]
cc

,Unnamed: 0,Labels,Predictions
41,41,33,12
227,227,33,12
234,234,33,12
492,492,33,12
947,947,33,12
1174,1174,33,12
1278,1278,33,12
1405,1405,33,12
1574,1574,33,12
2776,2776,33,12


In [31]:
a = np.array([[2,3,4,5],[1,2,3,4]])
b = a.reshape(-1)
a,b

(array([[2, 3, 4, 5],
        [1, 2, 3, 4]]), array([2, 3, 4, 5, 1, 2, 3, 4]))